In [45]:
import pandas as pd
import numpy as np
import time
from scipy.spatial import distance
from scipy.spatial import distance_matrix
from copy import deepcopy
import math

In [48]:
X_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_resampling3000_X.csv")
y_train=pd.read_csv("D:\\lab; signal processing\\forStudents\\medData\\train_resampling3000_y.csv")
X_train=X_train.drop(['robust_dis'],axis=1)
y_train=y_train.drop(['robust_dis'],axis=1)
y_train=y_train.drop(['Unnamed: 0'],axis=1)

In [49]:
train = {'feature0':X_train['feature0'],'feature1':X_train['feature1'],'feature2':X_train['feature2'],'feature3':X_train['feature3'],'feature4':X_train['feature4'],'label':y_train['label'],'indexofpixel':y_train['indexofpixel'],'sourceofpixel':y_train['sourceofpixel']}
train = pd.DataFrame(data=train)
train = pd.DataFrame.sample(train,frac=1)

In [60]:
start1=time.time()
data_size_per_slice = 10000
data_slice_number = math.ceil(len(train)/data_size_per_slice)
print("The tatal amount of data slices: %d"%(data_slice_number ))
print("                           ")

for j in range(data_slice_number):
    print("current data slice:"+str(j))
    train_slice = train[j*data_size_per_slice:(j+1)*data_size_per_slice]
    train_slice = pd.DataFrame.reset_index(train_slice).drop('index',axis=1)
    
    class1_indices=np.random.choice(train_slice[train_slice['label']==1].index,1,replace=False)
    class2_indices=np.random.choice(train_slice[train_slice['label']==2].index,1,replace=False)
    class1_sample = train_slice.iloc[class1_indices]
    class2_sample = train_slice.iloc[class2_indices]
    train_slice=train_slice.drop(class1_indices)
    train_slice=train_slice.drop(class2_indices)
    train_slice = pd.DataFrame.reset_index(train_slice).drop('index',axis=1)
    class1_sample = pd.DataFrame.reset_index(class1_sample).drop('index',axis=1)
    class2_sample = pd.DataFrame.reset_index(class2_sample).drop('index',axis=1)

    Store=list()
    newStore=list()
    class1_sample= {'feature0':class1_sample.iloc[0]['feature0'],'feature1':class1_sample.iloc[0]['feature1'],'feature2':class1_sample.iloc[0]['feature2'],'feature3':class1_sample.iloc[0]['feature3'],'feature4':class1_sample.iloc[0]['feature4'],'label':class1_sample.iloc[0]['label'],'indexofpixel':class1_sample.iloc[0]['indexofpixel'],'sourceofpixel':class1_sample.iloc[0]['sourceofpixel']}
    class2_sample= {'feature0':class2_sample.iloc[0]['feature0'],'feature1':class2_sample.iloc[0]['feature1'],'feature2':class2_sample.iloc[0]['feature2'],'feature3':class2_sample.iloc[0]['feature3'],'feature4':class2_sample.iloc[0]['feature4'],'label':class2_sample.iloc[0]['label'],'indexofpixel':class2_sample.iloc[0]['indexofpixel'],'sourceofpixel':class2_sample.iloc[0]['sourceofpixel']}
    newStore.append(class1_sample)
    newStore.append(class2_sample)
    
    start=time.time()
    count=0
    while newStore:
        count = count+1
        oldlength = len(Store)
        for element in newStore:
            Store.append(element)

        temp=pd.DataFrame(data=newStore)
        temp=np.array(temp)
        temp=temp[:,0:5]
        np.cast[float](temp)

        train_temp=np.array(train_slice)
        train_temp=train_temp[:,0:5]

        start1=time.time()
        dis=distance.cdist(temp.reshape(len(newStore),5),train_temp,'euclidean')
        nearest_q_matrix=dis.min(0)
        nearest_q_indexmat=dis.argmin(axis=0)

        #update the nearest distance of q in (T-S) dataset, and the corresponding p in S.
        if count==1:
            nearest_q={'p_index':nearest_q_indexmat,'nearest_dis':nearest_q_matrix}
            nearest_q=pd.DataFrame(data=nearest_q)
            nearest_q=pd.concat([nearest_q,train_slice['label']],axis=1,join='outer')
        else:
            nearest_q_indexmat=nearest_q_indexmat+oldlength
            nearest_q_new={'p_index_new':nearest_q_indexmat,'nearest_dis_new':nearest_q_matrix}
            nearest_q_new=pd.DataFrame(data=nearest_q_new)
            nearest_q=pd.concat([nearest_q,nearest_q_new],axis=1,join='outer')
            nearest_q['p_index'] = np.where((nearest_q['nearest_dis_new']<nearest_q['nearest_dis']),nearest_q['p_index_new'],nearest_q['p_index'])
            nearest_q['nearest_dis'] = np.where((nearest_q['nearest_dis_new']<nearest_q['nearest_dis']),nearest_q['nearest_dis_new'],nearest_q['nearest_dis'])
            nearest_q=nearest_q.drop('p_index_new',axis=1)
            nearest_q=nearest_q.drop('nearest_dis_new',axis=1)

        Store_dataframe=pd.DataFrame(data=Store)
        #attach p's label to the dataframe of nearest_q
        p_label=pd.DataFrame(Store_dataframe.iloc[nearest_q['p_index']]['label'])
        p_label.columns=['p_label']
        p_label=pd.DataFrame.reset_index(p_label).drop('index',axis=1)
        nearest_q=pd.concat([nearest_q,p_label],axis=1,join='outer')

        #choose those data  q whose label is not equal to the lapel of nearest_q
        # and then creat the dataframe called different_predict_points
        temp_setting=np.ones(len(train_slice))
        temp_setting=pd.DataFrame(data=temp_setting)
        temp_setting.columns=['setting']
        nearest_q=pd.concat([nearest_q,temp_setting],axis=1,join='outer')
        nearest_q['choose'] = np.where((nearest_q['label']!=nearest_q['p_label']),nearest_q['setting'],np.nan)

        different_predic_points=pd.DataFrame(nearest_q[nearest_q['choose']==1])

        nearest_q=nearest_q.drop('setting',axis=1)
        nearest_q=nearest_q.drop('choose',axis=1)


        newStore=list()

        start2=time.time()
        for i in range(len(Store)):
            temp=different_predic_points[different_predic_points['p_index']==i]
            temp=pd.DataFrame.reset_index(temp)
            if len(temp)>0 :
                location=temp['nearest_dis'].idxmin(axis=1)
                add_positon=int(temp.loc[location]['index'])
                temp_newone= {'feature0':train_slice.loc[add_positon]['feature0'],'feature1':train_slice.loc[add_positon]['feature1'],'feature2':train_slice.loc[add_positon]['feature2'],'feature3':train_slice.loc[add_positon]['feature3'],'feature4':train_slice.loc[add_positon]['feature4'],'label':train_slice.loc[add_positon]['label'],'indexofpixel':train_slice.loc[add_positon]['indexofpixel'],'sourceofpixel':train_slice.loc[add_positon]['sourceofpixel']}
                newStore.append(temp_newone)
                train_slice=train_slice.drop(add_positon)
                nearest_q=nearest_q.drop(add_positon)
            
        nearest_q=nearest_q.drop('p_label',axis=1)
        train_slice=pd.DataFrame.reset_index(train_slice).drop('index',axis=1)
        nearest_q=pd.DataFrame.reset_index(nearest_q).drop('index',axis=1)
    print("current count:%d current size of store:%d time:%5.1f minute"%(count,len(Store),(time.time()-start)/60))
    
    Store=pd.DataFrame(data=Store)
    if (j==0):
        Store_total=Store
    else:
        frame=[Store_total,Store]
        Store_total=pd.concat(frame)
        Store_total=pd.DataFrame.reset_index(Store_total).drop('index',axis=1)
print("The total execution time:%5.1f minute"%((time.time()-start1)/60))

The tatal amount of data slices: 32
                           
current data slice:0
current count:26 current size of store:2671 time:  0.8 minute
current data slice:1
current count:24 current size of store:2725 time:  0.7 minute
current data slice:2
current count:28 current size of store:2659 time:  0.8 minute
current data slice:3
current count:26 current size of store:2715 time:  0.8 minute
current data slice:4
current count:26 current size of store:2732 time:  0.9 minute
current data slice:5
current count:29 current size of store:2696 time:  1.0 minute
current data slice:6
current count:25 current size of store:2745 time:  0.8 minute
current data slice:7
current count:25 current size of store:2607 time:  0.7 minute
current data slice:8
current count:25 current size of store:2704 time:  0.7 minute
current data slice:9
current count:32 current size of store:2789 time:  0.9 minute
current data slice:10
current count:24 current size of store:2603 time:  0.6 minute
current data slice:11


In [61]:
Store_total

,feature0,feature1,feature2,feature3,feature4,indexofpixel,label,sourceofpixel
0,0.159176,-1.116775,-0.220058,-0.259830,0.083105,[37 26 25],1.0,4
1,-1.579836,0.196558,-0.035183,1.613370,-0.490321,[55 56 40],2.0,10
2,-0.108690,-0.972476,-0.307463,-0.270390,-0.160603,[34 26 35],2.0,5
3,-1.649910,-0.000336,-0.153631,1.319140,-0.617347,[47 42 31],1.0,10
4,0.452036,-0.920437,-0.164502,-0.012098,0.018728,[57 42 50],2.0,10
5,-1.737136,0.080353,0.298190,1.607374,-0.691765,[50 54 29],1.0,10
6,-0.065129,-0.875837,-0.293881,-0.232657,-0.100835,[36 43 48],1.0,3
7,-1.631593,-0.121762,-0.163148,1.419328,-0.581578,[50 53 51],2.0,10
8,0.045745,-1.068631,-0.197874,-0.098174,0.584036,[56 44 51],2.0,10
9,-1.948740,0.278323,-0.224037,1.813984,-0.498627,[27 51 31],1.0,7


In [74]:
Store_total_feature=Store_total.loc[:,['feature0','feature1','feature2','feature3','feature4']]

In [76]:
Store_total_label = Store_total.loc[:,['indexofpixel','label','sourceofpixel']]

In [77]:
Store_total_label

,indexofpixel,label,sourceofpixel
0,[37 26 25],1.0,4
1,[55 56 40],2.0,10
2,[34 26 35],2.0,5
3,[47 42 31],1.0,10
4,[57 42 50],2.0,10
5,[50 54 29],1.0,10
6,[36 43 48],1.0,3
7,[50 53 51],2.0,10
8,[56 44 51],2.0,10
9,[27 51 31],1.0,7


In [79]:
Store_total_feature.to_csv("D:\\lab; signal processing\\forStudents\\medData\\train_x_resample3000_FCNN_newOne.csv",index=False)
Store_total_label.to_csv("D:\\lab; signal processing\\forStudents\\medData\\train_y_resample3000_FCNN_newOne.csv",index=False)